In [1]:
import requests, json, re, ast, urllib.parse
from bs4 import BeautifulSoup
from markdownify import markdownify as md
from pathlib import Path
import nbformat
from nbformat.v4 import new_notebook, new_markdown_cell, new_code_cell


# ───────────── Helpers to Parse Inputs/Outputs ─────────────
def to_python_literal(val: str):
    val = val.strip()
    val = re.sub(r"\btrue\b", "True", val, flags=re.I)
    val = re.sub(r"\bfalse\b", "False", val, flags=re.I)
    val = re.sub(r"\bnull\b", "None", val, flags=re.I)
    try:
        return ast.literal_eval(val)
    except Exception:
        return val.strip('"').strip("'")


def parse_examples(example_blocks):
    parsed = []
    for block in example_blocks:
        im = re.search(r"Input:\s*(.+?)\s*$", block, flags=re.M)
        om = re.search(r"Output:\s*(.+?)\s*$", block, flags=re.M)
        if not im or not om:
            continue
        in_text, out_text = im.group(1), om.group(1)

        kwargs = {}
        if "=" not in in_text:
            kwargs["arg"] = to_python_literal(in_text)
        else:
            for piece in in_text.split(","):
                if "=" in piece:
                    k, v = piece.split("=", 1)
                    kwargs[k.strip()] = to_python_literal(v)
        parsed.append((kwargs, to_python_literal(out_text)))
    return parsed


# ───────────── Fetch LeetCode HTML / meta ─────────────
def fetch_leetcode_details(url):
    slug = urllib.parse.urlparse(url).path.strip("/").split("/")[-1]
    graphql_url = "https://leetcode.com/graphql"
    headers = {
        "Content-Type": "application/json",
        "Referer": f"https://leetcode.com/problems/{slug}/",
        "User-Agent": "Mozilla/5.0",
    }
    query = {
        "query": """
        query getQuestionDetail($titleSlug: String!) {
            question(titleSlug: $titleSlug) {
                title
                titleSlug
                content
                codeSnippets { lang langSlug code }
            }
        }
        """,
        "variables": {"titleSlug": slug},
    }
    question = requests.post(graphql_url, json=query, headers=headers).json()["data"]["question"]

    soup = BeautifulSoup(question["content"], "html.parser")

    # examples
    example_blocks_md = []
    for blk in soup.select("div.example-block"):
        in_ = blk.find("strong", string=re.compile("^Input"))
        out_ = blk.find("strong", string=re.compile("^Output"))
        in_val = in_.find_next("span").text if in_ else ""
        out_val = out_.find_next("span").text if out_ else ""
        example_blocks_md.append(f"Input: {in_val}\nOutput: {out_val}")
        blk.decompose()
    examples_parsed = parse_examples(example_blocks_md)

    # strip "Example 1:" etc.
    for hdr in soup.find_all("strong"):
        if re.match(r"Example\s*\d+", hdr.text.strip()):
            hdr.decompose()

    # constraints
    constraints = "-"
    ch = soup.find("strong", string="Constraints:")
    if ch:
        ul = ch.find_next("ul")
        if ul:
            constraints = "\n".join(f"- {' '.join(li.stripped_strings)}" for li in ul)
            ul.decompose()
        ch.decompose()

    return {
        "title": question["title"],
        "slug": question["titleSlug"],
        "content_md": md(str(soup)).strip(),
        "examples_md": example_blocks_md,
        "examples_parsed": examples_parsed,
        "constraints": constraints,
        "code": next(
            (s["code"] for s in question["codeSnippets"] if s["lang"] == "Python3"),
            "def solution(*args, **kwargs):\n    pass",
        ),
    }


def extract_function_name(code: str):
    m = re.search(r"def (\w+)\(", code)
    return m.group(1) if m else "solution"


# ───────────── Notebook Generator ─────────────
def create_notebook_from_leetcode_url(url):
    d = fetch_leetcode_details(url)
    fn_name = extract_function_name(d["code"])
    uses_class = "class Solution" in d["code"]
    call_target = f"Solution().{fn_name}" if uses_class else fn_name

    cells = [
        new_markdown_cell(f"# {d['title']}"),
        new_markdown_cell(f"## 🧠 Problem Statement\n\n{d['content_md']}\n\n---"),
        new_markdown_cell("## 📘 Example(s)\n\n```python\n"
                          + "\n\n".join(d["examples_md"]) + "\n```\n\n---"),
        new_markdown_cell(f"## 📏 Constraints\n\n{d['constraints']}\n\n---"),
        new_markdown_cell("## 💭 My Analysis\n\n- \n- \n- \n\n---"),
        new_markdown_cell("## 🛠️ Attempt(s)"),
        new_code_cell(d["code"]),
    ]

    # ── Friendly test-runner code ──
    tl = [
        "def _run_auto_tests(func):",
        "    tests = [",
    ]
    for kw, exp in d["examples_parsed"]:
        tl.append(f"        ({kw}, {exp}),")
    tl += [
        "    ]",
        "    all_passed = True",
        "    for i, (kw, expected) in enumerate(tests, 1):",
        "        try:",
        "            result = func(**kw) if isinstance(kw, dict) else func(kw)",
        "            if result == expected:",
        "                print(f'✅ Test {i} passed | Input: {kw} → Output: {result}')",
        "            else:",
        "                print(f'❌ Test {i} failed')",
        "                print(f'   Input:    {kw}')",
        "                print(f'   Expected: {expected}')",
        "                print(f'   Got:      {result}')",
        "                all_passed = False",
        "        except Exception as e:",
        "            print(f'❌ Test {i} crashed with exception: {e}')",
        "            print(f'   Input: {kw}')",
        "            all_passed = False",
        "    print('🎉 All tests passed!' if all_passed else '⚠ Some tests failed.')",
        "",
        f"_run_auto_tests({call_target})",
    ]

    cells.append(new_markdown_cell("## 🧪 Test Cases"))
    cells.append(new_code_cell("\n".join(tl)))
    cells.append(new_markdown_cell("## ✅ Final Version"))
    cells.append(new_code_cell(d["code"]))
    cells.append(new_markdown_cell("## 🧵 Time/Space Complexity\n\n- Time: \n- Space: "))

    nb = new_notebook(cells=cells)
    Path("leetcode_notebooks").mkdir(exist_ok=True)
    out_path = Path("leetcode_notebooks") / f"{d['slug']}.ipynb"
    with out_path.open("w", encoding="utf-8") as f:
        nbformat.write(nb, f)

    print(f"✅ Notebook saved to {out_path}")

In [2]:
create_notebook_from_leetcode_url("https://leetcode.com/problems/isomorphic-strings/")

✅ Notebook saved to leetcode_notebooks\isomorphic-strings.ipynb
